In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go

## Doden bij nucleare energy

In [2]:
df = pd.read_csv('nuclear_data/rates_death_from_energy_production_per_twh.csv')
df = df.sort_values(by='Deaths per TWh of electricity production', ascending=False)
df.head()

,Entity,Deaths per TWh of electricity production,Year
1,Brown coal,32.720,2021
2,Coal,24.620,2021
6,Oil,18.430,2021
0,Biomass,4.630,2021
3,Gas,2.821,2021


In [3]:
fig = px.bar(
    df,
    x='Entity',
    y='Deaths per TWh of electricity production',
    color='Deaths per TWh of electricity production',
    color_continuous_scale='RdYlGn_r'  # correcte naam
)

fig.update_layout(
    title="Death-rate per TWh produced per energy source",
    xaxis_title="Energy production method",
    yaxis_title="Deaths per TWh",
    coloraxis_showscale=False,
)

fig.show()

In [ ]:
df = pd.read_csv('impacts-of-energy-sources.csv')

df = df.drop(['Agricultural land use', 'Urban land use', 'Death rates', 'Year'], axis=1)

df = df.rename(columns={'Greenhouse gas emissions': 'Greenhouse gas emissions (kg CO2 eq.)', 
                        'Freshwater eutrophication': 'Freshwater eutrophication (kg P eq.)', 
                        'Ionising radiation':'Ionising radiation (kBq 235U eq)', 
                        'Water use':'Water use (m3)',
                        'Metal and mineral requirements':'Metal and mineral requirements (kg Sb eq.)',
                        'Non-carcinogenic toxicity':'Non-carcinogenic toxicity (CTUh)',
                        'Carcinogenic toxicity':'Carcinogenic toxicity (CTUh)',
                        'Total land use':'Total land use (points)',
                        'Uranium':'Uranium (tonnes)'})

df['Entity'] = df['Entity'].replace('Coal with carbon capture (CCS)', 'Coal')

df['Entity'] = df['Entity'].replace('Concentrating solar (tower)', 'Concentrating Solar')
df['Entity'] = df['Entity'].replace('Concentrating solar (trough)', 'Concentrating Solar')

df['Entity'] = df['Entity'].replace('Solar PV, cadmium (roof)', 'Solar PV')
df['Entity'] = df['Entity'].replace('Solar PV, cadmium (on-ground)', 'Solar PV')
df['Entity'] = df['Entity'].replace('Solar PV, silicon (on-ground)', 'Solar PV')
df['Entity'] = df['Entity'].replace('Solar PV, silicon (roof)', 'Solar PV')

df['Entity'] = df['Entity'].replace('Gas with carbon capture (CCS)', 'Gas')

df['Entity'] = df['Entity'].replace('Offshore wind', 'Wind')
df['Entity'] = df['Entity'].replace('Onshore wind', 'Wind')

# Groepeer en sommeer

df_combined = df.drop(columns=['Aluminium', 'Chromium', 'Cobalt', 'Copper', 'Manganese', 'Molybdenum', 'Nickel', 'Silicon', 'Zinc'])
df_combined['Human toxicity (CTUh)'] = df_combined['Non-carcinogenic toxicity (CTUh)'] + df_combined['Carcinogenic toxicity (CTUh)']
df_combined = df_combined.drop(columns=['Non-carcinogenic toxicity (CTUh)', 'Carcinogenic toxicity (CTUh)' ])
df_combined = df_combined[['Entity', 'Greenhouse gas emissions (kg CO2 eq.)', 
                           'Freshwater eutrophication (kg P eq.)', 
                           'Ionising radiation (kBq 235U eq)', 
                           'Water use (m3)', 
                           'Total land use (points)', 
                           'Human toxicity (CTUh)', 
                           'Metal and mineral requirements (kg Sb eq.)', 
                           'Uranium (tonnes)' ]]
df_combined = df_combined.groupby('Entity', as_index=False).sum()

In [ ]:
# Create a bar chart for each column (except Entity)
for column in df_combined.columns[1:]:
    fig = px.bar(df_combined, 
                 x='Entity', 
                 y=column,
                 title=f'{column} by Energy Source per the delivery of 1 kWh of electricity',
                 labels={'Entity': 'Energy Source', column: column},
                 color='Entity')
    
    # Improve layout for better readability
    fig.update_layout(
        xaxis_title='Energy Source',
        yaxis_title=column,
        height=600,
        width=800,
        title_font=dict(size=15),
        title_x=0.5
    )
    
    # Show the plot
    fig.show()